# H5 to MDA Converter for Mountainsort

- Takes in data that has already been preprocessed in h5_explorer (or h5_subtractor if subtracting probe data)
- Original paper was Chung et al. 2017 in Neuron. That version of mountainsort is already obsolete.
- Latest MS is here: https://github.com/flatironinstitute/mountainsort_examples/blob/master/docs/preparing_datasets.md

### User Input

In [1]:
datafile = "experiment_C20200330-175336_filtered_64.h5"
fs = 3000
chan_range = [0,64] # starting at channel 0, not channel 1
row_num = [21,20,21] # rows per column (# values should == # columns)

### Import Dependencies

In [3]:
from IPython.display import HTML
import numpy as np
from mlpy import mdaio
import pandas as pd
import h5py

### Read in the Multichannel Data
- Data has been pre-processed and already reflect the 0.195 datapoint/microvolt conversion from the willow system

In [4]:
f_data = h5py.File(datafile, 'r')
xs = f_data.get('sample_index') # channel # (x)
ys = f_data.get('channel_data') # actual data in microvolts (y)
print ("Data has " + str(ys.shape[0]) + " rows and " + str(ys.shape[1]) + " columns.")

Data has 1800000 rows and 64 columns.


### Make an M x N array
- M = # channels (rows)
- N = # timepoints (columns)

In [7]:
# Data will need to be transposed for mountainsort
mn = np.transpose(ys)
print ("Transposed data has " + str(mn.shape[0]) + " rows and " + str(mn.shape[1]) + " columns.")

# 32bit save function
mdaio.writemda32(ys[0:4, 60000:75000],'raw.mda')

#print(ys[0:30000, 0:4])

Transposed data has 64 rows and 1800000 columns.


True

### Make a probe map (1 shank)
- based off of probe maps designed in the original paper(Chung et al 2017 PMID: 28910621)
- As of April 2020 that version of MS is already obsolete, so map design may need to change for future versions
- Additional documentation from the latest version (in dev) is here https://github.com/flatironinstitute/mountainsort_examples/blob/master/docs/preparing_datasets.md
- #### Note that the mountainsort probe map is different than for kilosort
- #### Waiting on the final probe map from Jorg, so this is a best guess at the arrangement. Ch 42 (#41 in python notation) should be a dead channel (reserved for the wiring).

In [ ]:
arry = np.empty(shape = [ys.shape[1],2])
j = 0 # probe column #
row_val = -321
for chan in range(0, ys.shape[1]):
    if j == 0:
        arry[chan][0] = 0
        row_val = row_val + 15
        arry[chan][1] = row_val
    elif j == 1:
        arry[chan][0] = 22
        arry[chan][1] = row_val
    elif j == 2:
        arry[chan][0] = 44
        arry[chan][1] = row_val
        j = -1
    
    j = j + 1   

geom = pd.DataFrame(arry)
display(HTML(geom.to_html()))
geom.to_csv('geom.csv', index = False)
#f_shank1.close()